In [2]:
import os
import numpy as np
import pandas as pd
import glob
import re
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

from sktime.datasets import load_from_tsfile_to_dataframe
import warnings
from collections import Counter

warnings.filterwarnings('ignore')

In [3]:
filepath="/raid/hliu763/llama3_online/Time-Series-Library/MM_Dataset/RCW/RCW_TEST.ts"
df, labels = load_from_tsfile_to_dataframe(filepath, return_separate_X_and_y=True, replace_missing_vals_with='NaN')

# Count samples in each class
label_counts = Counter(labels)
min_count = min(label_counts.values())

# Create a dictionary to store indices for each class
class_indices = {label: np.where(labels == label)[0] for label in label_counts}

# Randomly select 'min_count' samples from each class
balanced_indices = []
for label, indices in class_indices.items():
    balanced_indices.extend(np.random.choice(indices, min_count, replace=False))

# Sort the indices to maintain the original order
balanced_indices.sort()

# Select the balanced data
balanced_df = df.iloc[balanced_indices].reset_index(drop=True)
balanced_labels = labels[balanced_indices]

# If the data needs to be reshaped, do it safely
reshaped_data = []
for idx in balanced_df.index:
    row_data = {col: balanced_df.loc[idx, col] for col in balanced_df.columns}
    reshaped_data.append(pd.DataFrame(row_data))

balanced_df = pd.concat(reshaped_data, axis=0).reset_index(drop=True)

df=balanced_df
labels=balanced_labels

In [4]:
labels = pd.Series(labels, dtype="category")
class_names = labels.cat.categories
labels_df = pd.DataFrame(labels.cat.codes,
                            dtype=np.int8)  # int8-32 gives an error when using nn.CrossEntropyLoss


In [5]:
def safe_len(x):
            if isinstance(x, (list, np.ndarray)):
                return len(x)
            elif isinstance(x, (float, int)):
                return 1  # 或者您可能想要返回 0，取决于您的需求
            else:
                return len(str(x))  # 作为后备选项，将其他类型转换为字符串

lengths = df.applymap(safe_len).values